## Census Batch Geocoding

See [https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf](https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf) for more information.

The Census batch geocoder is a great solution when you need to geocode high quantities of addresses located in the United States. We can submit a file containing up to 10,000 addresses at a time. The file needs to be formatted correctly with the following columns: Unique ID, Street address, City, State, ZIP. We do this preprocessing below, and subsequently write the CSV to file. We then send a GET request for the file using the `curl` utility to get the locations of the addresses. Change the variable names to fit your file format.

In [41]:
import pandas as pd
from tqdm import tqdm

addfile='RI_Independent_Schools.csv' #Input file with addresses
df = pd.read_csv(addfile)

ID = 'org_ID'
NAME = 'name'
ADDRESS = 'location_address1'
STATE = 'location_state'
CITY = 'location_city'
ZIP = 'location_zip'
MAX_LOCATIONS = 5

def split_dataframe(df, chunk_size=10000):
    return [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

def preprocess(df: pd.DataFrame) -> pd.DataFrame | list[pd.DataFrame]:
    df[ADDRESS] = df[ADDRESS].str.replace('One', '1')
    if len(df) > 10000:
        df = split_dataframe(df)
    return df

batch_compatible_df = df[[ID, ADDRESS, CITY, STATE, ZIP]]
batch_compatible_df.to_csv(f'Batch_Compatible_{addfile}', index=False)

We specify a benchmark parameter, which is a numerical ID or name which references the version of the locator to be
used. This generally corresponds to address locators based on MAF/TIGER benchmarks, otherwise known as TIGER/Line geography. At the time of writing, there were three benchmarks available:
- 4: Public Address Ranges - Current Benchmark
- 8: Public Address Ranges - ACS2024 Benchmark
- 2020: Public Address Ranges - Census 2020 Benchmark

The differences are usually minimal; we will stick to the default here.

In [42]:
!curl --form addressFile=@Batch_Compatible_RI_Independent_Schools.csv --form benchmark=4 \
https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output Batch_Compatible_RI_Independent_Schools_Matched.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13043  100  9694  100  3349   7236   2500  0:00:01  0:00:01 --:--:--  9740


## Census Individual Geocoding

The Census Batch geocoder defined above should be your go-to in most cases: it runs much faster and provides easy access to results. However, in some cases the format specified above may not be a good fit for the data attributes you have. You may also want more control over the process, or to look at the matches for an individual address. In this case, individual geocoding may be a better choice. Here, we will use the Census API's One Line Address feature to lookup addresses built from individual rows. One Line Address lookup can match based on just two fields, the ADDRESS and ZIP fields. 

First, we define a function which will query the API for the Street, City, Address, and Zip provided in each row. We default to the current benchmark.

In [44]:
#can provide: (street, zip), (street, city, state) or more information.
import requests, urllib, traceback

address = '201 Thayer St,Providence,RI'
lineaddr_url = f'https://geocoding.geo.census.gov/geocoder/locations/onelineaddress?'

addfile='RI_Independent_Schools.csv' #Input file with addresses
df = pd.read_csv(addfile)

def get_request_census(address: str, url: str) -> list[dict]:
    address = urllib.parse.quote_plus(address)
    add_url = f'address={address}&benchmark=4&format=json'
    data_url = f'{lineaddr_url}{add_url}'
    #print(data_url)
    response=requests.get(data_url)
    return response.json()['result']['addressMatches']

Here, we construct the address string from the ADDRESS, CITY, STATE, and ZIP columns defined above. We send a GET request for each row of the dataframe. We retrieve best matching address, x, and y coordinates unless there are no matches. We append four columns to the dataframe specifying the new information. If there were no matches, we set them to None instead.

In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        address = f'{row[ADDRESS]},{row[CITY]},{row[STATE]},{row[ZIP]}'
        request_data = get_request_census(address, lineaddr_url)
        if len(request_data) > 0:
            matches = []
            for m in request_data:
                matches.append([row[ID], row[NAME], 
                                    m['matchedAddress'], 
                                    m['coordinates']['x'],
                                    m['coordinates']['y']])
            matches.sort(key=lambda x: x[2], reverse=True)
            best_match = matches[0]
            match_ct = len(request_data)
            match_address,x,y = best_match[2:5]
        else:
            score,match_address,x,y = None,None,None, None
        df.loc[idx, 'matches'] = match_ct
        df.loc[idx, 'match_address'] = match_address
        df.loc[idx, 'x'] = x
        df.loc[idx,'y'] = y

    except Exception as e:
        traceback.print_exc() 
df